In [1]:
import pandas as pd
import sqlite3

In [2]:
try:
    connection = sqlite3.connect("../data/checking-logs.sqlite")
    print("Successfully connected to database")
except FileNotFoundError as e:
    print("File not found")
    raise e

Successfully connected to database


In [3]:
try:
    delete_table = "DROP TABLE datamart"

    create_datamart_query = """
    CREATE TABLE datamart AS
    SELECT
        c.uid,
        c.labname,
        c.timestamp as first_commit_ts,
        MIN(p.datetime) as first_view_ts
    FROM checker c
    LEFT JOIN pageviews p ON c.uid = p.uid
    WHERE c.status = 'ready'
        AND c.numTrials = 1
        AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
        AND c.uid LIKE 'user_%'
    GROUP BY c.uid, c.labname, c.timestamp
    """

    connection.execute(delete_table)
    connection.execute(create_datamart_query)
    connection.commit()
    print("Creating datamart table")


except sqlite3.OperationalError as e:
    print("Database error")
    raise e

Creating datamart table


In [4]:
datamart = pd.read_sql("SELECT * FROM datamart", connection)

# Преобразуем timestamp колонки в datetime64[ns]
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])

# Сохраняем обратно в базу с правильными типами данных
datamart.to_sql('datamart', connection, if_exists='replace', index=False)

print(f"Lenght of datamart {len(datamart)}")
print("\nFirst rows")
print(datamart.head())
print("\nData types:")
print(datamart.dtypes)
print(datamart.info())


Lenght of datamart 140

First rows
      uid  labname            first_commit_ts              first_view_ts
0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3  user_1   laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4  user_1  laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136

Data types:
uid                        object
labname                    object
first_commit_ts    datetime64[ns]
first_view_ts      datetime64[ns]
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64

In [5]:
#Создаем test группу (пользователи с first_view_ts)
print("\nСоздаем test группу")
test = datamart[datamart['first_view_ts'].notna()].copy()

# Создаем control группу (пользователи без first_view_ts)
print("\nСоздаем control группу")
control = datamart[datamart['first_view_ts'].isna()].copy()

# Вычисляем среднее first_view_ts из test группы
avg_first_view_ts = test['first_view_ts'].mean()
print(f"Среднее first_view_ts из test группы: {avg_first_view_ts}")

# Заменяем пропущенные значения в control группе на среднее
print("\nЗаполняем пропущенные значения")
control['first_view_ts'] = control['first_view_ts'].fillna(avg_first_view_ts)

print("\nTest группа (первые 3 строки):")
print(test.head(3))
print("\nControl группа (первые 3 строки):")
print(control.head(3))

print(test.info())
print(control.info())


Создаем test группу

Создаем control группу
Среднее first_view_ts из test группы: 2020-04-27 00:40:05.761783552

Заполняем пропущенные значения

Test группа (первые 3 строки):
      uid  labname            first_commit_ts              first_view_ts
0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136

Control группа (первые 3 строки):
        uid   labname            first_commit_ts                 first_view_ts
12  user_11    laba05 2020-05-03 21:06:55.970293 2020-04-27 00:40:05.761783552
13  user_11  project1 2020-05-03 23:45:33.673409 2020-04-27 00:40:05.761783552
14  user_12    laba04 2020-04-18 17:07:51.767358 2020-04-27 00:40:05.761783552
<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           -------

In [6]:
try:
    test.to_sql('test', connection, if_exists='replace', index=False)
    control.to_sql('control', connection, if_exists='replace', index=False)

except sqlite3.OperationalError as e:
    print("Database error")
    raise e


print("\nTables check")
test_check = pd.read_sql("SELECT COUNT(*) as count FROM test", connection)
control_check = pd.read_sql("SELECT COUNT(*) as count FROM control", connection)

print(f"Строк в таблице test: {test_check['count'].iloc[0]}")
print(f"Строк в таблице control: {control_check['count'].iloc[0]}")


connection.close()


Tables check
Строк в таблице test: 59
Строк в таблице control: 81
